In [ ]:
import numpy as np
import pandas as pd
from rdkit import Chem
#Read substructures and datasets
datasets = pd.read_excel('ML-improve.xlsx') #Not provided
Dipyrone = datasets[datasets['Types of contaminants'] == 'Q']
#Dipyrone = datasets[datasets['Type of MB'] == 'RO'] If you want to study RO or NF, remove the previous # and the label above #
substructure = pd.read_csv('New Match.txt',header = None)
concatenated = pd.concat([Dipyrone.iloc[:, 7:27]], axis=1)
arr3 = concatenated.to_numpy()

In [ ]:
#Match all substructures of SMILES (and record the quantity)
arr1 =[[0 for x in range(len(substructure))] for y in range(len(datasets))]
smis =datasets.SMILES
i = 0
j = 0
for smi in smis:
    mol = Chem.MolFromSmiles(smi)
    for sub in substructure[0].values :
        subMol = Chem.MolFromSmarts(sub)
        matches = mol.GetSubstructMatches(subMol)
        if len(matches):
            arr1[i][j] = len(matches)
            if sub =='c': 
                arr1[i][j] = arr1[i][j]/6
        j = j+1
    i= i+1
    j=0

In [ ]:
arr = np.hstack((arr1, arr3))


X = arr
y = Dipyrone.iloc[:, 27]


import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn import metrics

#Divide the training set and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)#37

#Initialize the model
model = xgb.XGBRegressor()

#Set hyperparameter candidate values
param_grid = {
    'colsample_bytree': [0.3,0.4,0.5,0.6,0.7,0.8,0.9],
    'learning_rate': [0.1, 0.01],
    'max_depth': [3,4, 5,6,7,8,9,10],
    'alpha': [10, 20, 30,40,50,60,70],
    'n_estimators': [10,20,30,40 ,50,60,70,80,90,100]
}

#Create GridSearchCV object
grid_search = GridSearchCV(model, param_grid, scoring='r2', cv=5,n_jobs = -1)

#Perform grid search
grid_search.fit(X_train, y_train)

#Output the optimal hyperparameter combination
print("Best parameters found: ", grid_search.best_params_)

#Use the model with optimal parameters for prediction
best_model = grid_search.best_estimator_
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)
print('training R2:', metrics.r2_score(y_train,y_train_pred))
print('testing R2:', metrics.r2_score(y_test,y_test_pred))

In [ ]:
#Find the optimal random_state

import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn import metrics

#Parameter candidate values - obtained from the above training, the following is a randomly given set of parameters
param_grid = {
    'colsample_bytree': [0.6],
    'learning_rate': [0.1],
    'max_depth': [7],
    'alpha': [10],
    'n_estimators': [100]
}                                                                            

#Initialize variables to store the best model and the highest R2 score
best_r2 = -np.inf
best_model = None
best_random_state = None


for random_state in range(160):

    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
    
   
    model = xgb.XGBRegressor()
    
    
    grid_search = GridSearchCV(model, param_grid, scoring='r2', cv=5, n_jobs=-1)
    
    
    grid_search.fit(X_train, y_train)
    
    
    y_test_pred = grid_search.best_estimator_.predict(X_test)
    test_r2 = metrics.r2_score(y_test, y_test_pred)
    
    #If the R2 score of the current model is higher than the previous best score, update the optimal model and score
    if test_r2 > 0.8:
        best_r2 = test_r2
        best_model = grid_search.best_estimator_
        best_random_state = random_state
        y_train_pred = best_model.predict(X_train)
        y_test_pred = best_model.predict(X_test)
        print('Best random_state:', best_random_state)
        print('training R2:', metrics.r2_score(y_train, y_train_pred))
        print('test R2:', metrics.r2_score(y_test, y_test_pred))


y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# 输出训练和测试 R2 分数
print("Best parameters found: ", grid_search.best_params_)
print('Best random_state:', best_random_state)
print('training R2:', metrics.r2_score(y_train, y_train_pred))
print('testing R2:', metrics.r2_score(y_test, y_test_pred))

In [ ]:
from sklearn import metrics
train_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
print('training RMSE:', train_rmse)
print('testing RMSE:', test_rmse)

In [ ]:
import shap
#Initialize SHAP interpreter using the model with optimal parameters
explainer = shap.TreeExplainer(best_model)

#Calculate SHAP value
shap_values = explainer.shap_values(X_train)

#Retrieve the name of each row in the substructure and convert it into a list
substructure_names = substructure[0].tolist()
feature_names = []

#Add the name of the original feature
for col_name in concatenated.columns:
    feature_names.append(col_name)

# Add the infrastructure name, and original feature name to feature_name
feature_names = substructure_names  + feature_names

shap.summary_plot(shap_values, X_train, feature_names=feature_names)

In [ ]:
import shap

explainer = shap.TreeExplainer(best_model)

#Calculate SHAP value
shap_values = explainer.shap_values(X_train)

#Convert to Explanation Object
shap_explainer = shap.Explanation(shap_values, feature_names=feature_names)

#Draw a bar chart of SHAP feature importance
shap.plots.bar(shap_explainer)

In [ ]:
#Calculate the average absolute SHAP value for each feature
mean_abs_shap = np.abs(shap_explainer.values).mean(axis=0)

#Construct a DataFrame containing feature names and corresponding average absolute SHAP values
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Mean Abs SHAP Value': mean_abs_shap
})

#Export the results as an Excel file
importance_df.to_excel("shap-DMF-MRL model.xlsx", index=False)

In [ ]:
#Molecular fingerprint analysis
sub = shap_values[:,:12]
shap.summary_plot(sub, X_train[:, :12],feature_names=substructure_names) 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# SHAP PDP using C6H6 as an example
feature_index = feature_names.index("-C6H6")


shap_vals_for_c6h6 = shap_values[:, feature_index]


feature_vals_for_c6h6 = X_train[:, feature_index]  # 如果 X_train 是 DataFrame


plt.scatter(feature_vals_for_c6h6, shap_vals_for_c6h6, alpha=0.7)
plt.xlabel("Feature value: -C6H6")
plt.ylabel("SHAP value for -C6H6")
plt.title("SHAP Dependence for -C6H6")
plt.show()


# create DataFrame
df_export = pd.DataFrame({
    "feature_value(-C6H6)": feature_vals_for_c6h6,
    "shap_value(-C6H6)": shap_vals_for_c6h6
})

# output Excel 
output_filename = "shap-C6H6 scattered data.xlsx"
df_export.to_excel(output_filename, index=False)


In [ ]:
#Only export the true and predicted values of the target variable, without including features
train_results = pd.DataFrame({
    "y_true": y_train.values,
    "y_pred": y_train_pred
})
test_results = pd.DataFrame({
    "y_true": y_test.values,
    "y_pred": y_test_pred
})

#Use ExcelWriter to save two DataFrames separately to different worksheets
with pd.ExcelWriter("xgboost.xlsx") as writer:
    train_results.to_excel(writer, sheet_name="Train", index=False)
    test_results.to_excel(writer, sheet_name="Test", index=False)